In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## CNN
http://cs231n.github.io/

## RNN
http://web.stanford.edu/class/cs224n/

https://www.youtube.com/watch?v=vT1JzLTH4G4&list=PL3FW7Lu3i5JvHM8ljYj-zLfQRF3EO8sYv

## Siraj Raval
https://www.youtube.com/channel/UCWN3xxRkmTPmbKwht9FuE5A/videos

In [15]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

batch_size = 64
test_batch_size = 10000

learning_rate = 0.01

epochs = 4

log_interval = 16

In [4]:
device

device(type='cpu')

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True)

Processing...
Done!


In [19]:
? nn.Linear

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [10]:
model = Net().to(device)

In [22]:
? F.dropout

In [11]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.5)

In [13]:
def train(epoch):
    model.train() #sets module to training mode
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data) #forward pass
#         loss = F.nll_loss(output, target) #negative log likelihood
        loss = F.cross_entropy(output, target) 
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [14]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad(): #Disabling gradient calc is useful for inference, reduces memory consumption
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [16]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.383605
Train Epoch: 1 [1024/60000 (2%)]	Loss: 2.307981
Train Epoch: 1 [2048/60000 (3%)]	Loss: 2.258795
Train Epoch: 1 [3072/60000 (5%)]	Loss: 2.251578
Train Epoch: 1 [4096/60000 (7%)]	Loss: 2.220585
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.183869
Train Epoch: 1 [6144/60000 (10%)]	Loss: 2.134310
Train Epoch: 1 [7168/60000 (12%)]	Loss: 2.062712
Train Epoch: 1 [8192/60000 (14%)]	Loss: 1.830346
Train Epoch: 1 [9216/60000 (15%)]	Loss: 1.691538
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.339271
Train Epoch: 1 [11264/60000 (19%)]	Loss: 1.242369
Train Epoch: 1 [12288/60000 (20%)]	Loss: 1.208603
Train Epoch: 1 [13312/60000 (22%)]	Loss: 1.068041
Train Epoch: 1 [14336/60000 (24%)]	Loss: 1.354424
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.136357
Train Epoch: 1 [16384/60000 (27%)]	Loss: 0.984626
Train Epoch: 1 [17408/60000 (29%)]	Loss: 0.906940
Train Epoch: 1 [18432/60000 (31%)]	Loss: 0.776091
Train Epoch: 1 [19456/60000 (32%)]	Loss: 0.815874
Train Epoch: 1 [204

Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.350005
Train Epoch: 3 [47104/60000 (78%)]	Loss: 0.643657
Train Epoch: 3 [48128/60000 (80%)]	Loss: 0.292666
Train Epoch: 3 [49152/60000 (82%)]	Loss: 0.305131
Train Epoch: 3 [50176/60000 (84%)]	Loss: 0.147939
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.391049
Train Epoch: 3 [52224/60000 (87%)]	Loss: 0.207508
Train Epoch: 3 [53248/60000 (89%)]	Loss: 0.272734
Train Epoch: 3 [54272/60000 (90%)]	Loss: 0.197940
Train Epoch: 3 [55296/60000 (92%)]	Loss: 0.237534
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.320474
Train Epoch: 3 [57344/60000 (96%)]	Loss: 0.303955
Train Epoch: 3 [58368/60000 (97%)]	Loss: 0.256656
Train Epoch: 3 [59392/60000 (99%)]	Loss: 0.451116

Test set: Average loss: 0.0870, Accuracy: 9725/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.177527
Train Epoch: 4 [1024/60000 (2%)]	Loss: 0.362940
Train Epoch: 4 [2048/60000 (3%)]	Loss: 0.331656
Train Epoch: 4 [3072/60000 (5%)]	Loss: 0.256799
Train Epoch: 4 [4096/60000 (7%)]	Loss: 0.224397
Tr

In [10]:
? torch.no_grad